# В этом файле лежат вспомогательные куски кода

Это создание первичного датафрейма 

In [46]:
import pandas as pd
import re

data = {'name': 
['Ахматова', 'Бродский','Лермонтов', 'Маяковский', 'Пушкин', 'Тютчев'], 
        'path': 
['Ahmatova', 'Brodskiy', 'Lermontov', 'Mayakovskiy', 'Pushkin', 'Tyutchev']} 
 
df = pd.DataFrame.from_dict(data)
texts = []

for index, row in df.iterrows():
    value = row['path']
    with open(f'/books/{value}.txt') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]\n\n','',text)
        texts.append(text)

df['text'] = texts
df.to_csv('base.csv')  # в этот файл потом дозаписываем во время работы бота

In [44]:
df.columns.values

array(['Unnamed: 0', 'name', 'path', 'text'], dtype=object)

Дальше код для краулера, которым с я сайта *Культура* скачала список русский писателей (см. про модерацию)

In [3]:
import requests
from pprint import pprint
session = requests.session()


from fake_useragent import UserAgent


ua = UserAgent()
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://www.culture.ru/literature/persons?page=', headers=headers)

from bs4 import BeautifulSoup
import re

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:102.0) Gecko/20100101 Firefox/102.0'}


In [10]:
greats = []
href_plus = ['1', '2', '3', '4', '5', '6']

for page in href_plus:
    url_o = 'https://www.culture.ru/literature/persons?page=' + page

    req = session.get(url_o, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    title = soup.find_all('div', {'class': "Rosqt"})
    greats = greats + title

In [35]:
greats_list = []

for i in range(len(greats)):
    greats_list.append(greats[i].text)

In [37]:
with open('greats_list.txt', 'w', encoding='UTF-8') as outfile:
    print(','.join(greats_list), file=outfile)  # отсюда читаем список в коде бота

А здесь была отладка функции, рисующей облако

In [27]:
import pandas as pd

import matplotlib.pyplot as plt
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re

import collections
from collections import Counter

from pymystem3 import Mystem

df = pd.read_csv('C:/Users/ulyas/OneDrive/Рабочий стол/прожект/base.csv', encoding='utf=8')
authors = df['name'].tolist()
path = df['path'].tolist()
poem = df['text'].tolist()

def clouds_func(name):
    nltk.download('punkt')
    nltk.download('stopwords')
    stop_words = stopwords.words('russian')

    text = poem[authors.index('Ахматова')]
    text = re.sub(r'[^\w\s]\n\n','',text)

    word_tokenize(text)
    words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    words = [w for w in words if w not in stop_words]

    top = dict(Counter(words).most_common(30))
    all_top = list(top.keys())

    stop_words = stop_words + all_top
    words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    words = [w for w in words if w not in stop_words]

    m = Mystem()
    lemmas = ''.join(m.lemmatize(' '.join(words)))

    wordcloud = WordCloud(width = 400, height = 400,
                          background_color ='white', colormap = 'PiYG').generate(lemmas)

    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 

    plt.savefig(f'wordcloud_{name}.jpg')
    plt.show()

    return wordcloud